Word Document Formatter

1. importing libraries

In [12]:
from langgraph.graph import StateGraph
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from typing import TypedDict, Annotated, List, Dict, Optional, Literal, IO
from langgraph.graph import add_messages
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
model = ChatOpenAI(
    model="mistralai/devstral-2512:free",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key="",
)

2.states define

In [15]:
class DocStructure(BaseModel):
    document_type: str = Field(
        description="Type of document: academic, research, business, general"
    )

    headings: Dict[str, int] = Field(
        description="Detected headings mapped to heading level (e.g., {'Introduction': 1})"
    )

    body_font_size: int = Field(
        description="Target font size for body text"
    )

    heading_font_size: int = Field(
        description="Target font size for headings"
    )

    alignment: str = Field(
        description="Text alignment: left, right, center, justify"
    )

    issues_found: List[str] = Field(
        description="Formatting or structural issues detected"
    )


In [16]:
from enum import Enum

class ActionEnum(str, Enum):
    SET_FONT_SIZE = "set_font_size"
    SET_ALIGNMENT = "set_alignment"
    ALIGN_TEXT = "align_text"
    FORMAT_AS_LIST = "format_as_list"
    REMOVE_EMPTY_PARAGRAPHS = "remove_empty_paragraphs"

# class TargetEnum(str, Enum):
#     HEADING = "heading"
#     SUBHEADING = "subheading"
#     BODY_TEXT = "body_text"
#     KEY_POINTS = "key_points"
#     LIST_ITEMS = "list_items"
#     DOCUMENT = "document"
#     END_OF_DOCUMENT = "end_of_document"

class FormatCommand(BaseModel):
    action: ActionEnum
    target: str
    value: int | str   
    source: Literal["auto", "user"]


In [17]:
class FormatCommandList(BaseModel):
    commands: List[FormatCommand]

In [18]:
def last_write(_, new):
    return new

In [19]:
class wordFormatter(TypedDict, total=False):
    doc: str
    document: Document
    structDoc: Annotated[list, last_write]
    AutoDetect: Annotated[DocStructure, last_write]
    AutoDetectCmd: Annotated[list, last_write]
    userCmd: Annotated[list, last_write]
    formatCmd: Annotated[list, last_write] 
    output_doc: Optional[str]
    output_pdf: Optional[str]
    user_instruction: str

3. nodes Building

NodeFn1: Loading the document

In [20]:
from docx import Document

def load_docx(path: str):
    doc = Document(path)
    paragraphs = []

    for p in doc.paragraphs:
        paragraphs.append({
            "text": p.text.strip(),
            "style": p.style.name if p.style else None,
            "alignment": p.alignment,
            "runs": [
                {
                    "text": r.text,
                    "bold": r.bold,
                    "italic": r.italic,
                    "font_size": r.font.size.pt if r.font.size else None
                }
                for r in p.runs
            ]
        })
    return paragraphs


In [21]:
loader = load_docx('Draft.docx')
# doc = loader.load()
print(loader)

[{'text': 'Introduction', 'style': 'Heading 1', 'alignment': None, 'runs': [{'text': 'Introduction', 'bold': None, 'italic': None, 'font_size': None}]}, {'text': 'Qwertyujnbvcddx', 'style': 'List Paragraph', 'alignment': None, 'runs': [{'text': 'Qwertyujnbvcddx', 'bold': None, 'italic': None, 'font_size': None}]}, {'text': 'Ertyhhbvcx', 'style': 'List Paragraph', 'alignment': None, 'runs': [{'text': 'Ertyhhbvcx', 'bold': None, 'italic': None, 'font_size': None}]}, {'text': 'sdfgbhcd', 'style': 'List Paragraph', 'alignment': None, 'runs': [{'text': 'sdfgbhcd', 'bold': None, 'italic': None, 'font_size': None}]}, {'text': 'This document is created for a word processing formatting project. The purpose of this file is to allow the user to apply formatting such as text justification, font size, font color, line spacing, and paragraph alignment.', 'style': 'Normal', 'alignment': None, 'runs': [{'text': 'This ', 'bold': None, 'italic': None, 'font_size': None}, {'text': 'document is created fo

In [22]:
def loadDocNode(state: wordFormatter) -> dict:
    doc = Document(state["doc"])
    return {
        "document": doc,
        "structDoc": load_docx(state["doc"])
    }


NodeFn2: Structure Analysis Agent node

In [23]:
def docStructureNode(state: wordFormatter) -> dict:
    struct_doc = state["structDoc"]

    parser = PydanticOutputParser(pydantic_object=DocStructure)

    prompt = PromptTemplate(
        template="""
You are a document structure analysis agent.

Analyze the following document structure and produce a formatting plan.

Document structure:
{struct_doc}

{format_instructions}
""",
        input_variables=["struct_doc"],
        partial_variables={
            "format_instructions": parser.get_format_instructions()
        }
    )

    response = model.invoke(
        prompt.format(struct_doc=struct_doc)
    )

    return {
        "AutoDetect": parser.parse(response.content)
    }

NodeFn2.1 :converting to formatcmd

In [24]:
ACTION_DOC = """
Allowed actions (field `action`):
- "set_font_size": Change font size for the specified target.
- "set_alignment": Change paragraph alignment.
- "align_text": Align only paragraphs containing specific text.
- "format_as_list": Convert paragraphs into a bulleted list.
- "remove_empty_paragraphs": Remove trailing empty paragraphs.


Allowed target (field `target`):
- anything present in the document can also be the target
- "heading": All heading-style paragraphs (Heading 1–3).
- "subheading": Subsections under headings.
- "body_text": Normal body paragraphs.
- "key_points": Paragraphs marked or inferred as key points.
- "list_items": Existing bullet or numbered list items.
- "document": Entire document.
- "end_of_document": Only trailing paragraphs at the end.


Return float values as integer not as string
Do NOT invent new actions like "justify", "bolden", etc.
"""

In [25]:
cmd_parser = PydanticOutputParser(pydantic_object=FormatCommandList)


def AutoDetectCommandNode(state: wordFormatter) -> dict:
    instruction = state["AutoDetect"]  

    prompt = PromptTemplate(
        template="""
You are a document formatting planner.

Given these issues:
{issues}
{action_doc}

Generate formatting commands.
Return them strictly in this format:
{format_instructions}
""",
        input_variables=["issues"],
        partial_variables={"format_instructions": cmd_parser.get_format_instructions(),
                           "action_doc": ACTION_DOC,}
    )

    response = model.invoke(prompt.format(issues=instruction.issues_found))
    parsed = cmd_parser.parse(response.content)

    return {"AutoDetectCmd": parsed.commands}


NodeFn3: User Instructions node

In [26]:
cmd_parser = PydanticOutputParser(pydantic_object=FormatCommandList)



def chatCommandNode(state: wordFormatter) -> dict:
    instruction = state["user_instruction"]

    prompt = PromptTemplate(
        template="""
        You are a document formatting planner
        Given the user instruction
        Instruction: {instruction}
        {action_doc}
        Generate formatting commands.
        Return them strictly in this format:
        {format_instructions}
        """,
        input_variables=["instruction"],
        partial_variables={"format_instructions": cmd_parser.get_format_instructions(),
                           "action_doc": ACTION_DOC,}

    )
    response = model.invoke(prompt.format(instruction = instruction))
    parsed = cmd_parser.parse(response.content)
    
    return {
        "userCmd": parsed.commands
    }

In [27]:
TARGET_SCOPE = {
    "document": 0,      # global
    "text": 0,
    "body_text": 1,
    "heading": 2,
    "list_items": 2,
}

In [28]:
TARGET_PRIORITY = {
    "heading": 1,
    "subheading": 2,
    "body_text": 3,
    "key_points": 4,
    "list_items": 5,
    "document": 6,
    "end_of_document": 7,
    # any other targets can be added here
}

In [29]:
def mergeCommandNode(state: wordFormatter) -> dict:
    auto_cmd = state.get("AutoDetectCmd") or []
    user_cmd = state.get("userCmd") or []

    if not isinstance(auto_cmd, list):
        raise TypeError(f"AutoDetectCmd must be list, got {type(auto_cmd)}")

    if not isinstance(user_cmd, list):
        raise TypeError(f"userCmd must be list, got {type(user_cmd)}")

    resolved = {}

    def scope(t): 
        return TARGET_SCOPE.get(t, 0)

    def should_override(existing, incoming):
        if incoming.source == "user" and existing.source == "auto":
            return True
        return scope(incoming.target) >= scope(existing.target)

    for cmd in auto_cmd + user_cmd:
        key = cmd.action
        if key not in resolved or should_override(resolved[key], cmd):
            resolved[key] = cmd

    return {"formatCmd": list(resolved.values())}



NodeFn4: ToolcmdNode

In [30]:
def set_font_size_tool(doc, target: str, value: int):
    """Logic for font size"""
    for p in doc.paragraphs:
        if target == "heading" and "Heading" in p.style.name:
            for r in p.runs: r.font.size = Pt(int(value))
        elif target == "body_text" and "Normal" in p.style.name:
            for r in p.runs: r.font.size = Pt(int(value))
        elif target == "document":
            for r in p.runs: r.font.size = Pt(int(value))
    return f"Set {target} font to {value}"

def set_alignment_tool(doc, target: str, value: str):
    """Logic for alignment"""
    align_map = {
        "left": WD_ALIGN_PARAGRAPH.LEFT,
        "center": WD_ALIGN_PARAGRAPH.CENTER,
        "right": WD_ALIGN_PARAGRAPH.RIGHT,
        "justify": WD_ALIGN_PARAGRAPH.JUSTIFY,
    }
    for p in doc.paragraphs:
        if target == "document" or target.lower() in p.text.lower():
            p.paragraph_format.alignment = align_map.get(value.lower(), p.paragraph_format.alignment)

    return f"Set {target} alignment to {value}"

In [31]:
# Map your FormatCommand.action strings to the functions


TOOL_REGISTRY = {
    "set_font_size": set_font_size_tool,
    "set_alignment": set_alignment_tool,
    "align_text": set_alignment_tool # Alias
}



def ToolcmdNode(state: wordFormatter) -> dict:
    doc = state["document"]
    commands = state.get("formatCmd", [])
    
    for cmd in commands:
        func = TOOL_REGISTRY.get(cmd.action.value)
        if func:
            print(f"Applying Action: {cmd.action} -> {cmd.target}")
            # Execute the tool
            func(doc, cmd.target, cmd.value)
        else:
            print(f"No tool found for action: {cmd.action}")

    output_path = "Final_Formatted.docx"
    doc.save(output_path)
    return {"output_doc": output_path}


4.Graph Struct

In [32]:
graph = StateGraph(wordFormatter)

graph.add_node("load_docx", loadDocNode)
graph.add_node("doc_structure", docStructureNode)
graph.add_node("chat_command", chatCommandNode)
graph.add_node("AutoDetect_command", AutoDetectCommandNode)
graph.add_node("merge_command", mergeCommandNode)
graph.add_node('Toolcmd', ToolcmdNode)



graph.set_entry_point("load_docx")

graph.add_edge("load_docx", "doc_structure")
graph.add_edge("load_docx", "chat_command")
graph.add_edge("doc_structure", "AutoDetect_command")

graph.add_edge("chat_command", "merge_command")
graph.add_edge("AutoDetect_command", "merge_command")

graph.add_edge("merge_command", "Toolcmd")
graph.set_finish_point("Toolcmd")


app = graph.compile()


In [36]:
app.get_graph().print_ascii()

ImportError: Install grandalf to draw graphs: `pip install grandalf`.

In [ ]:
# result = app.invoke({
#      "doc": "Draft.docx",
#      "structDoc": [],
#      "formatPlan": [],
#      "output_doc": []
#  })

# print(result["output_doc"])
# print(result["structDoc"])
# print(result["formatPlan"])


In [37]:
res = app.invoke({
    "doc": "Draft.docx",
    "user_instruction": "Justify doc"
})

AuthenticationError: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}

In [ ]:
print(res['formatCmd'])

[FormatCommand(action=<ActionEnum.SET_FONT_SIZE: 'set_font_size'>, target='heading', value=12, source='auto'), FormatCommand(action=<ActionEnum.FORMAT_AS_LIST: 'format_as_list'>, target='list_items', value='bullet', source='auto'), FormatCommand(action=<ActionEnum.REMOVE_EMPTY_PARAGRAPHS: 'remove_empty_paragraphs'>, target='end_of_document', value='all', source='auto'), FormatCommand(action=<ActionEnum.ALIGN_TEXT: 'align_text'>, target='Key points:', value='left', source='auto'), FormatCommand(action=<ActionEnum.SET_ALIGNMENT: 'set_alignment'>, target='document', value='justify', source='user')]


In [ ]:
print(res.get("AutoDetect"))

document_type='academic' headings={'Introduction': 1, 'Project Description': 1, 'Conclusion': 1} body_font_size=12 heading_font_size=14 alignment='left' issues_found=["Inconsistent font sizes in headings (e.g., 'Project Description' has font size 10.0)", 'Inconsistent font sizes in body text (e.g., some parts have font size 18.0 or 14.0)', "List items ('Qwertyujnbvcddx', 'Ertyhhbvcx', 'sdfgbhcd') are not properly formatted as a list", 'Multiple empty paragraphs at the end of the document', "Some text is centered without clear justification (e.g., 'Key points:' and related items)"]


In [ ]:
print(res.get("AutoDetectCmd"))

[FormatCommand(action=<ActionEnum.SET_FONT_SIZE: 'set_font_size'>, target='heading', value=12, source='auto'), FormatCommand(action=<ActionEnum.SET_FONT_SIZE: 'set_font_size'>, target='body_text', value=12, source='auto'), FormatCommand(action=<ActionEnum.FORMAT_AS_LIST: 'format_as_list'>, target='list_items', value='bullet', source='auto'), FormatCommand(action=<ActionEnum.REMOVE_EMPTY_PARAGRAPHS: 'remove_empty_paragraphs'>, target='end_of_document', value='all', source='auto'), FormatCommand(action=<ActionEnum.ALIGN_TEXT: 'align_text'>, target='Key points:', value='left', source='auto')]


In [ ]:
print(res.get("userCmd"))

[FormatCommand(action=<ActionEnum.SET_ALIGNMENT: 'set_alignment'>, target='document', value='justify', source='user')]
